In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/pfm/')

# Pytorch libraries
import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# Internal files
import config
import dataloader
import models
from baseline import get_model, save_checkpoint

import math
import os
import sys
import time
from collections import OrderedDict
import random

import pandas as pd


noise file noisy_label_kv.txt generated with noise: 0.5



In [3]:
# Get the original_dataset
loader = dataloader.KeyDataLoader()
train_loader, valid_loader, test_loader = loader.run()

# Hyper Parameter settings
random.seed(config.seed)
# torch.cuda.set_device(config.gpuid)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
use_cuda = torch.cuda.is_available()

# Networks setup
print('\nModel setup')
print('| Building network: {}'.format(config.model))
net = get_model()

if use_cuda:
  net.cuda()

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:704: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")



Model setup
| Building network: resnet50


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
def load_checkpoint(epoch):
  checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + "_" + str(epoch) + '.pth.tar')
  net.load_state_dict(checkpoint['state_dict'])

In [0]:
def test(loader):
    net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = net(inputs).cuda()

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Grab validation results
    test_acc = 100. * correct/total

    return(round(float(test_acc),2))

In [0]:
def get_accy_list(max_epoch, loader):
  max_epoch = 101

  accy_list = []
  for epoch in range(1,101):
    load_checkpoint(epoch)
    accy_list.append(test(loader))
  return(accy_list)
    # print(accy_list)

In [0]:
print("computing test list\n")
test_list  = get_accy_list(101, test_loader)
print(test_list)
print("computing val list\n")
valid_list = get_accy_list(101, valid_loader)
print(valid_list)
print("computing train list\n")
train_list = get_accy_list(101, train_loader)

df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

computing test list



In [0]:
clean_kv = config.drive_dir +"/plots/"+ config.checkpoint+".csv"
# clean_kv = "/content/drive/My Drive/Colab_Notebooks/pfm/data/asdf.txt"
df.to_csv(clean_kv, sep=' ', index=False,header = False)

In [0]:
train_list